In [ ]:
# https://www.ncbi.nlm.nih.gov/biosample/SAMN36419051/
# https://www.ncbi.nlm.nih.gov/datasets/docs/v2/api/rest-api/
# https://www.ncbi.nlm.nih.gov/datasets/docs/v2/api/api-keys/#get-your-api-key

import os
import requests
import pprint

import panadas as pd
from rich import inspect
from dotenv import load_dotenv

dotenv_path = "/home/chase/asp_workshop_keys"
load_dotenv(dotenv_path=dotenv_path)

NCBI_KEY = os.getenv("NCBI_KEY", "your_key_here")  

In [26]:
class Organism:
    def __init__(self, genome_id):
        self.id = genome_id
        self.data = self._get_data()
        
    def _get_json(self, url, api_key=None):
        headers = {'accept': 'application/json'}
        if api_key:
            headers["api-key"] = api_key
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching data from: {e}")
            return None
        
    def _get_data(self):
        url = f"https://api.ncbi.nlm.nih.gov/datasets/v2/genome/accession/{self.id}/dataset_report"
        return self._get_json(url, api_key=NCBI_KEY)
    
    def what_am_i(self):
        print(f'I am a: "{self.data['reports'][0]['organism']['common_name']}"')


In [27]:
a=Organism(genome_id = "GCF_000001635")
a.what_am_i()

I am a: "house mouse"


In [28]:
a.data

{'reports': [{'accession': 'GCF_000001635.27',
   'current_accession': 'GCF_000001635.27',
   'paired_accession': 'GCA_000001635.9',
   'source_database': 'SOURCE_DATABASE_REFSEQ',
   'organism': {'tax_id': 10090,
    'organism_name': 'Mus musculus',
    'common_name': 'house mouse',
    'infraspecific_names': {'strain': 'C57BL/6J'}},
   'assembly_info': {'assembly_level': 'Chromosome',
    'assembly_status': 'current',
    'paired_assembly': {'accession': 'GCA_000001635.9', 'status': 'current'},
    'assembly_name': 'GRCm39',
    'assembly_type': 'haploid',
    'bioproject_lineage': [{'bioprojects': [{'accession': 'PRJNA20689',
        'title': 'Genome sequence finishing for Mus musculus, currently maintained by the Genome Reference Consortium (GRC)'}]}],
    'bioproject_accession': 'PRJNA20689',
    'release_date': '2020-06-24',
    'description': 'Genome Reference Consortium Mouse Build 39',
    'submitter': 'Genome Reference Consortium',
    'refseq_category': 'reference genome',
 

In [29]:
a=Organism("GCF_038242915")
a.what_am_i()

KeyError: 'common_name'

######## 2 ########


In [10]:
class Organism:
    def __init__(self, genome_id):
        self.id = genome_id
    def what_am_i(self):
        url = f"https://api.ncbi.nlm.nih.gov/datasets/v2/genome/accession/{self.id}/dataset_report"
        try:
            response = requests.get(url, headers={'accept': 'application/json', "api-key": NCBI_KEY})
            response.raise_for_status()
            data = response.json()
            if 'common_name' in data['reports'][0]['organism']:
                print(f"I am a: {data['reports'][0]['organism']['common_name']}")
            elif 'organism_name' in data['reports'][0]['organism']:
                print(f"🤓 I am a: {data['reports'][0]['organism']['organism_name']}")
            else:
                print("I don't know")
        except requests.RequestException as e:
            print(f"Error fetching data from NCBI: {e}")

In [11]:
a=Organism("GCF_000001635")
a.what_am_i()

I am a: house mouse


In [12]:
a=Organism("GCF_038242915")
a.what_am_i()

🤓 I am a: Streptomyces albidoflavus


######## 3 ########

In [13]:
class Organism:
    def __init__(self, genome_id):
        self.id = genome_id
    @staticmethod
    def _extract_taxid(data):
        try:
            return data['reports'][0]['organism']['tax_id']
        except (KeyError, IndexError):
            return None
    @staticmethod
    def _extract_assembly_name(data):
        try:
            return data['reports'][0]['assembly_info']['assembly_name']
        except (KeyError, IndexError):
            return None
    def set_info(self):
        url = f"https://api.ncbi.nlm.nih.gov/datasets/v2/genome/accession/{self.id}/dataset_report"
        try:
            response = requests.get(url, headers={'accept': 'application/json', "api-key": NCBI_KEY})
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Error fetching data from NCBI: {e}")
        # assign taxid
        self.taxid = self._extract_taxid(response.json())
        # assign assembly_name
        self.assembly_name = self._extract_assembly_name(response.json())

In [14]:
a = Organism("GCF_000001635.27")

In [15]:
a.taxid


AttributeError: 'Organism' object has no attribute 'taxid'

In [16]:
a.assembly_name

AttributeError: 'Organism' object has no attribute 'assembly_name'

In [17]:
a.set_info()
a.taxid
a.assembly_name

'GRCm39'

######## 4 ########

In [18]:
class Organism:
    __slots__ = ['id', 'BLAST_LINK', 'GDV_LINK', 'FTP_LINK', 'ASSEMBLY_PUBMED', 'ASSEMBLY_NUCCORE', 'ASSEMBLY_NUCCORE_REFSEQ', 'ASSEMBLY_NUCCORE_GENBANK', 'CGV_LINK']
    def __init__(self, genome_id):
        self.id = genome_id
        self.get_outlinks()
    def get_outlinks(self):
        """Fetches assembly links from NCBI and assigns them to the instance attributes."""
        url = f"https://api.ncbi.nlm.nih.gov/datasets/v2/genome/accession/{self.id}/links"
        try:
            response = requests.get(url, headers={'accept': 'application/json', "api-key": NCBI_KEY})
            response.raise_for_status()
            data = response.json()
            for i in data['assembly_links']:
                try:
                    link_type = i['assembly_link_type']
                    if link_type in self.__slots__:
                        setattr(self, link_type, i['resource_link'])
                except KeyError as e:
                    print(f"Missing key in assembly link data: {e}")
        except requests.RequestException as e:
            print(f"Error fetching data from NCBI: {e}")

In [19]:
a = Organism("GCF_000001635.27")

In [20]:
inspect(Organism, methods=True)

╭────────────────────────────────────────── <class '__main__.Organism'> ──────────────────────────────────────────╮
│ class Organism(genome_id):                                                                                      │
│                                                                                                                 │
│         ASSEMBLY_NUCCORE = <member 'ASSEMBLY_NUCCORE' of 'Organism' objects>                                    │
│ ASSEMBLY_NUCCORE_GENBANK = <member 'ASSEMBLY_NUCCORE_GENBANK' of 'Organism' objects>                            │
│  ASSEMBLY_NUCCORE_REFSEQ = <member 'ASSEMBLY_NUCCORE_REFSEQ' of 'Organism' objects>                             │
│          ASSEMBLY_PUBMED = <member 'ASSEMBLY_PUBMED' of 'Organism' objects>                                     │
│               BLAST_LINK = <member 'BLAST_LINK' of 'Organism' objects>                                          │
│                 CGV_LINK = <member 'CGV_LINK' of 'Organism' objects>                                            │
│                 FTP_LINK = <member 'FTP_LINK' of 'Organism' objects>                                            │
│                 GDV_LINK = <member 'GDV_LINK' of 'Organism' objects>                                            │
│                       id = <member 'id' of 'Organism' objects>                                                  │
│             get_outlinks = def get_outlinks(self): Fetches assembly links from NCBI and assigns them to the     │
│                            instance attributes.                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [21]:
inspect(a, methods=True)

╭────────────────────────────────────────── <class '__main__.Organism'> ──────────────────────────────────────────╮
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <__main__.Organism object at 0x7613dfe5e420>                                                                │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│         ASSEMBLY_NUCCORE = 'https://www.ncbi.nlm.nih.gov/nuccore/?from_uid=7358741&linkname=assembly_nuccore_r… │
│ ASSEMBLY_NUCCORE_GENBANK = 'https://www.ncbi.nlm.nih.gov/nuccore/?from_uid=7358741&linkname=assembly_nuccore_i… │
│  ASSEMBLY_NUCCORE_REFSEQ = 'https://www.ncbi.nlm.nih.gov/nuccore/?from_uid=7358741&linkname=assembly_nuccore_r… │
│          ASSEMBLY_PUBMED = 'https://www.ncbi.nlm.nih.gov/nuccore/?from_uid=7358741&linkname=assembly_pubmed'    │
│               BLAST_LINK = 'https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&PROG_DEF=blastn&BLA… │
│                 CGV_LINK = 'https://www.ncbi.nlm.nih.gov/cgv/GCF_000001635.27'                                  │
│                 FTP_LINK = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39'   │
│                 GDV_LINK = 'https://www.ncbi.nlm.nih.gov/genome/gdv/browser/genome/?id=GCF_000001635.27'        │
│                       id = 'GCF_000001635.27'                                                                   │
│             get_outlinks = def get_outlinks(): Fetches assembly links from NCBI and assigns them to the         │
│                            instance attributes.                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [22]:
a.BLAST_LINK

'https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&PROG_DEF=blastn&BLAST_SPEC=GDH_GCF_000001635.27'

# Taxonomy

In [ ]:
curl -X POST "https://api.ncbi.nlm.nih.gov/datasets/v2/taxonomy/taxon_suggest" \
 -H 'accept: application/json'\
 -H 'content-type: application/json' \
 -d '{"taxon_query":"hum"}' 

In [40]:
try:
    response = requests.post(
        "https://api.ncbi.nlm.nih.gov/datasets/v2/taxonomy/taxon_suggest",
        headers={'accept': 'application/json', 'content-type': 'application/json'},
        json={"taxon_query": "Streptomyces", "tax_rank_filter": "higher_taxon", "exact_match": True}
    )
    response.raise_for_status()
    taxon = response.json()
    print(taxon)
except requests.RequestException as e:
    print(f"Error fetching data from NCBI: {e}")

{'sci_name_and_ids': [{'sci_name': 'Streptomyces', 'tax_id': '1883', 'matched_term': 'Streptomyces', 'rank': 'GENUS'}]}


In [42]:
# "https://api.ncbi.nlm.nih.gov/datasets/v2/taxonomy/taxon/1883/dataset_report" 

taxid = taxon['sci_name_and_ids'][0]['tax_id']

try: 
    response = requests.get(
        f"https://api.ncbi.nlm.nih.gov/datasets/v2/taxonomy/taxon/{taxid}/dataset_report",
        headers={'accept': 'application/json', "api-key": NCBI_KEY}
    )
    response.raise_for_status()
    data = response.json()
    pprint.pprint(data)
except requests.RequestException as e:
    print(f"Error fetching data from NCBI: {e}")


{'reports': [{'query': ['1883'],
              'taxonomy': {'children': [89304,
                                        2593676,
                                        60449,
                                        1464011,
                                        1517690,
                                        1433417,
                                        1521032,
                                        1520577,
                                        1433415,
                                        1045501,
                                        2053010,
                                        1073330,
                                        476556,
                                        1818004,
                                        1300267,
                                        1981977,
                                        68286,
                                        1042984,
                                        156453,
                                        6544

Using WikiData

![images/wikidata-wiki.png](images/wikidata-wiki.png)

In [44]:
import requests
import pprint

WIKIDATA_SPARQL_URL = "https://query.wikidata.org/sparql"
HEADERS = {"Accept": "application/sparql-results+json"}

def run_sparql_query(query: str) -> dict:
    """Run a SPARQL query and return the JSON result."""
    response = requests.get(WIKIDATA_SPARQL_URL, headers=HEADERS, params={"query": query})
    response.raise_for_status()
    return response.json()

def get_entity_id_by_label(label: str, language: str = "en") -> str:
    """
    Finds the Wikidata entity ID for a given label (name).
    """
    query = f"""
    SELECT ?item WHERE {{
      ?item rdfs:label "{label}"@{language} .
    }}
    LIMIT 1
    """
    data = run_sparql_query(query)
    if "results" not in data or "bindings" not in data["results"]:
        raise ValueError(f"Unexpected response format from Wikidata for label '{label}': {data}")
    bindings = data["results"]["bindings"]
    if not bindings:
        raise ValueError(f"No Wikidata entity found for label '{label}'")
    if "item" not in bindings[0] or "value" not in bindings[0]["item"]:
        raise ValueError(f"Missing 'item' or 'value' in SPARQL result for label '{label}': {bindings[0]}")
    entity_url = bindings[0]["item"]["value"]
    return entity_url.rsplit("/", 1)[-1]

def get_species_taxon(entity_id: str) -> str:
    """
    Returns the taxon entity ID with rank 'species' (wd:Q7432) if available.
    """
    query = f"""
    SELECT ?taxon ?taxonLabel WHERE {{
      wd:{entity_id} wdt:P703 ?taxon .
      ?taxon wdt:P105 wd:Q7432 .  # Filter: only taxon with rank = species
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 1
    """
    data = run_sparql_query(query)
    if "results" not in data or "bindings" not in data["results"]:
        raise ValueError(f"Unexpected response format from Wikidata for entity {entity_id}: {data}")
    bindings = data["results"]["bindings"]
    if not bindings:
        raise ValueError(f"No species-level taxon found for entity {entity_id}")
    if "taxon" not in bindings[0] or "value" not in bindings[0]["taxon"]:
        raise ValueError(f"Missing 'taxon' or 'value' in SPARQL result for entity {entity_id}: {bindings[0]}")
    taxon_url = bindings[0]["taxon"]["value"]
    return taxon_url.rsplit("/", 1)[-1]


def get_entity_properties(entity_id: str, limit: int = 300) -> dict:
    """Get a dictionary of property → value labels for a given Wikidata entity."""
    query = f"""
    SELECT ?propertyLabel ?valueLabel WHERE {{
      wd:{entity_id} ?prop ?value .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }} LIMIT {limit}
    """
    return run_sparql_query(query)

def get_entity_label(entity_id: str) -> str:
    """
    Returns the label (name) of a Wikidata entity.
    """
    query = f"""
    SELECT ?itemLabel WHERE {{
      wd:{entity_id} rdfs:label ?itemLabel .
      FILTER (lang(?itemLabel) = "en")
    }}
    LIMIT 1
    """
    data = run_sparql_query(query)
    if "results" not in data or "bindings" not in data["results"]:
        raise ValueError(f"Unexpected response format from Wikidata for entity {entity_id}: {data}")
    bindings = data["results"]["bindings"]
    if not bindings:
        raise ValueError(f"No label found for entity {entity_id}")
    if "itemLabel" not in bindings[0] or "value" not in bindings[0]["itemLabel"]:
        raise ValueError(f"Missing 'itemLabel' or 'value' in SPARQL result for entity {entity_id}: {bindings[0]}")
    return bindings[0]["itemLabel"]["value"]

In [45]:
# Step 0: Find the Wikidata ID for "Surugamide A"
source_entity_name = "Surugamide A"
source_entity_id = get_entity_id_by_label(source_entity_name)
print(f"The Wikidata ID for '{source_entity_name}' is: {source_entity_id}")

print(f"\nLooking up taxon for Wikidata entity: {source_entity_name} ({source_entity_id})")

# Step 1: Get the taxon entity linked via "found in taxon" (P703)
taxon_entity_id = get_species_taxon(source_entity_id)
print(f"Found in taxon entity ID: {taxon_entity_id}")

# Step 2: Get the label for the taxon entity
taxon_entity_label = get_entity_label(taxon_entity_id)
print(f"Found in taxon entity name: {taxon_entity_label}")


# Step 3: Get properties of the taxon entity
taxon_properties = get_entity_properties(taxon_entity_id)

print("\nTaxon properties:")
pprint.pprint(taxon_properties)

The Wikidata ID for 'Surugamide A' is: Q77372486

Looking up taxon for Wikidata entity: Surugamide A (Q77372486)
Found in taxon entity ID: Q22286833
Found in taxon entity name: Streptomyces albidoflavus

Taxon properties:
{'head': {'vars': ['propertyLabel', 'valueLabel']},
 'results': {'bindings': [{'valueLabel': {'type': 'literal',
                                          'value': '2248300457'}},
                          {'valueLabel': {'type': 'literal',
                                          'value': '2024-09-14T23:06:01Z'}},
                          {'valueLabel': {'type': 'literal',
                                          'value': 'specie de bacterie'}},
                          {'valueLabel': {'type': 'literal',
                                          'value': 'Art der Gattung '
                                                   'Streptomyces'}},
                          {'valueLabel': {'type': 'literal',
                                          'value': 'vrsta bakte

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Result:
    compoundLabel: str = field(default="")
    pubchemCID: str = field(default="")
    inchiKey: str = field(default="")
    foundInTaxonLabel: str = field(default="")
    bioactivityLabel: str = field(default="")
    
    def __post_init__(self):
        self.compoundLabel = self.compoundLabel or "Unknown Compound"
        self.pubchemCID = self.pubchemCID or "N/A"
        self.inchiKey = self.inchiKey or "N/A"
        self.foundInTaxonLabel = self.foundInTaxonLabel or "Unknown Taxon"
        self.bioactivityLabel = self.bioactivityLabel or "N/A"



def get_compounds_found_in_taxon(taxon_entity: str, limit: int = 100) -> dict:
    query = """SELECT ?compoundLabel ?pubchemCID ?inchiKey ?foundInTaxonLabel  ?bioactivityLabel WHERE {
  # Find compounds that have "found in taxon" of Streptomyces lavendulae (Q1144013)
  ?compound wdt:P703 wd:Q1144013 .

  # Include the found taxon in the results (it will always be Q1144013 in this case)
  BIND(wd:Q1144013 AS ?foundInTaxon)

  # Optional external identifiers
  OPTIONAL { ?compound wdt:P662 ?pubchemCID. } # PubChem CID
  OPTIONAL { ?compound wdt:P235 ?inchiKey. }   # InChIKey

  # Using P2868 (drug used for treatment) and P2175 (medical condition treated) as examples.
  OPTIONAL { ?compound wdt:P2868 ?bioactivity. }
  OPTIONAL { ?compound wdt:P2175 ?bioactivity. }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
    """
    data = run_sparql_query(query)
    for result in data['results']['bindings']:
        yield Result(
            compoundLabel=result.get('compoundLabel', {}).get('value', ""),
            pubchemCID=result.get('pubchemCID', {}).get('value', ""),
            inchiKey=result.get('inchiKey', {}).get('value', ""),
            foundInTaxonLabel=result.get('foundInTaxonLabel', {}).get('value', ""),
            bioactivityLabel=result.get('bioactivityLabel', {}).get('value', "")
        )

data = get_compounds_found_in_taxon("Q1144013")

In [127]:
data.__next__()

Result(compoundLabel='rifampicin', pubchemCID='5381226', inchiKey='JQXXHWHPUNPDRT-WLSIYKJHSA-N', foundInTaxonLabel='Streptomyces', bioactivityLabel='antibiotic')

In [129]:
data = get_compounds_found_in_taxon("Q1144013")

In [130]:
df = DataFrame(data)

In [131]:
df


,compoundLabel,pubchemCID,inchiKey,foundInTaxonLabel,bioactivityLabel
0,rifampicin,5381226,JQXXHWHPUNPDRT-WLSIYKJHSA-N,Streptomyces,antibiotic
1,rifampicin,135398735,JQXXHWHPUNPDRT-WLSIYKJHSA-N,Streptomyces,antibiotic
2,rifampicin,136262914,JQXXHWHPUNPDRT-WLSIYKJHSA-N,Streptomyces,antibiotic
3,rifampicin,5381226,JQXXHWHPUNPDRT-WLSIYKJHSA-N,Streptomyces,essential medicine
4,rifampicin,135398735,JQXXHWHPUNPDRT-WLSIYKJHSA-N,Streptomyces,essential medicine
...,...,...,...,...,...
95,riboflavin,493570,AUNGANRZJHBGPY-SCRDCRAPSA-N,Streptomyces,essential medicine
96,riboflavin,493570,AUNGANRZJHBGPY-SCRDCRAPSA-N,Streptomyces,vitamin B
97,riboflavin,493570,AUNGANRZJHBGPY-SCRDCRAPSA-N,Streptomyces,photosensitizer
98,riboflavin,493570,AUNGANRZJHBGPY-SCRDCRAPSA-N,Streptomyces,primary metabolite
